In [ ]:
import yfinance as yf
from datetime import datetime
import warnings

# Suprimir avisos específicos
warnings.filterwarnings("ignore", category=FutureWarning, module="yfinance")

def get_data_history_yahoo(symbol : str, period : str, interval, start = '1900-01-01', end = datetime.now(), prepost : bool = True, debug : bool = True):
    '''
    Data collection from yahoo

    __Parameters:__
    period : str
        Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max Either Use period parameter or use start and end
    interval : str
        Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo Intraday data cannot extend last 60 days
    start: str
        Download start date string (YYYY-MM-DD) or _datetime, inclusive. Default is 1900-01-01 E.g. for start="2020-01-01", the first data point will be on "2020-01-01"
    end: str
        Download end date string (YYYY-MM-DD) or _datetime, exclusive. Default is now E.g. for end="2023-01-01", the last data point will be on "2022-12-31"
    prepost : bool
        Include Pre and Post market data in results? Default is False
    debug: bool
        If passed as False, will suppress error message printing to console.
    '''
    dt = yf.Ticker(symbol).history(period=period, interval=interval, start=start, end=end, prepost=prepost, debug=debug)

    return dt

dt = get_data_history_yahoo(symbol="AAPL", period= "1d", interval = "1mo", start = "2024-10-01", end = datetime.now(), prepost = True, debug = True)


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-10-01 00:00:00-04:00,229.267761,237.229003,221.086760,225.661728,930736000,0.00,0.0
2024-11-01 00:00:00-04:00,220.727162,237.548652,219.468552,237.069183,891640600,0.25,0.0
2024-12-01 00:00:00-05:00,237.270004,250.800003,237.160004,248.664993,351079869,0.00,0.0


In [15]:
# Obter o objeto do ticker
ticker_data = yf.Ticker("AAPL")

# Informações gerais
info = ticker_data.info
info.keys

HTTPError: 401 Client Error: Unauthorized for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AAPL?modules=summaryProfile%2CfinancialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&ssl=true

Exemplo de Informações Retornadas:

longName: Nome completo do ativo.
sector: Setor da empresa.
industry: Indústria.
country: País.
currency: Moeda em que o ativo é negociado.
marketCap: Capitalização de mercado.
dividendYield: Rendimento por dividendo.
trailingPE: P/E (Price/Earnings) histórico.
forwardPE: P/E projetado.
beta: Beta do ativo.
52WeekChange: Mudança percentual em 52 semanas.
website: Website da empresa.

In [ ]:
# Dividendos
dividends = ticker_data.dividends
print("Dividendos:", dividends)

# Splits
splits = ticker_data.splits
print("Splits:", splits)

Obtenha informações sobre dividendos pagos e divisões de ações.

In [ ]:
# Recomendações de analistas
recommendations = ticker_data.recommendations
print(recommendations)

Obtenha a visão de analistas sobre o ativo.

In [ ]:
# Notícias
news = ticker_data.news
print(news)

Acessar notícias relacionadas ao ativo (se disponíveis).

In [ ]:
# Eventos corporativos
calendar = ticker_data.calendar
print(calendar)

Inclui datas de divulgação de resultados.

In [ ]:
# Datas disponíveis para opções
options_dates = ticker_data.options
print("Datas de opções:", options_dates)

# Obter opções para uma data específica
options = ticker_data.option_chain(options_dates[0])
print("Opções de compra (calls):", options.calls)
print("Opções de venda (puts):", options.puts)

Obtenha informações sobre opções de compra e venda (calls e puts).

In [ ]:
# Demonstrativo financeiro
financials = ticker_data.financials
print("Demonstrativo financeiro:", financials)

# Balanço patrimonial
balance_sheet = ticker_data.balance_sheet
print("Balanço patrimonial:", balance_sheet)

# Fluxo de caixa
cashflow = ticker_data.cashflow
print("Fluxo de caixa:", cashflow)

Informações financeiras mais detalhadas da empresa.

In [ ]:
# Dados de sustentabilidade
sustainability = ticker_data.sustainability
print(sustainability)

Informações de ESG (Environmental, Social, Governance).

In [ ]:
# Ações em circulação
shares = ticker_data.shares
print("Ações em circulação:", shares)

Obtenha informações sobre ações em circulação e histórico de splits.

In [ ]:
# Principais investidores institucionais
holders = ticker_data.institutional_holders
print("Instituições investidoras:", holders)


Dados sobre investidores institucionais e sua participação.